In [1]:
import pandas as pd
import numpy as np
import re
import os
from functools import reduce

In [2]:
# 先讀取爬蟲下來的資料透過function處理
# 目標 => 資料各欄位格式正確、沒有缺值
'''
1. 時間改為西元年(O)
2. 所有object欄位去除空白值(O)
3. 更改市場名稱
4. 交易價格&成交量欄位補缺值
5. 轉換型態
'''

def df_cleaner(file_path):
    # step 1 讀取資料(選取需求檔案)
    df = pd.read_csv(file_path, usecols=['日期', '市場', '平均價(元/公斤)', '交易量(公斤)'])
    
    # step 2 清洗與補缺值
    ## 時間 -> 日期改成西元年(設成index)
    df["日期"] = df["日期"].apply(lambda x: re.sub("\d{3}", "{}".format((int(x.split("/")[0]) + 1911)), x))
    df["日期"] = pd.to_datetime(df["日期"])
    # df = df.set_index("日期")
    
    ## 去除成交量中的雜質(數據型態為object都處理)
    target_cols = list(df.select_dtypes("object"))
    df[target_cols] = df[target_cols].apply(lambda x: x.str.strip(" "))
    
    # 更改市場名稱
    df["市場"] = df["市場"].apply(lambda x:x.split(" ")[1])
    
    ## 找出缺值(re.sub) & 轉換型態
    try:
        df["平均價(元/公斤)"] = df["平均價(元/公斤)"].apply(lambda x:re.sub(",|-", "", x)).replace("", np.nan).fillna(method="ffill").astype("float")
        df["交易量(公斤)"] = df["交易量(公斤)"].apply(lambda x:re.sub(",|-", "", x)).replace("", np.nan).fillna(method="ffill").astype("float")
    except:
        pass
    
    return df

In [3]:
# 乾淨資料進行resample
# 目標 => 各市場資料補成每日資料
'''
1. 依市場組成資料(台北一、台北二、三重、台中、鳳山)
2. 篩選欄位 & 重新命名
3. resample資料
4. 新增欄位(前日價格 & 5日移動平均)
5. 輸出檔案
'''

def df_merger(df, df_same, df_sub):
    
    # 合併欄位(依照日期)
    for market in markets:
        df_market =  df.loc[df["市場"] == market]
        same = df_same.loc[df_same["市場"] == market]
        sub = df_sub.loc[df_sub["市場"] == market]
        
        dfs = [df_market, same, sub]
        df_merged = reduce(lambda left,right: pd.merge(left, right, on="日期", how="left"), dfs)
        
        print(f"{market} 原始資料筆數(合併後): {df_merged.shape[0]}")
        
        # 將日期設為index
        df_merged.set_index("日期",inplace=True)
        
        # 篩選出必要欄位&重新命名
        cols = list(df_merged.columns) 
        df_merged = df_merged[cols[1:3] + [cols[4]] + [cols[7]]]
        df_merged.columns = [f"{product}_平均價", f"{product}_交易量", f"{same_type}_平均價", f"{substitution}_平均價"]
           
        # resample補齊每日資料(補值: 插值 -> 前值 -> 後值)
        df_merged = df_merged.resample("D").interpolate().fillna(method="ffill").fillna(method="bfill").applymap(lambda x: round(x,1))
        
        # 新增欄位(前日價格 & 5日移動平均)
        df_merged[f"{product}_前日平均價"] = df_merged[f"{product}_平均價"].shift(1).fillna(method="bfill")
        df_merged[f"{product}_5日平均價"] = round(df_merged[f"{product}_平均價"].rolling(5).mean().fillna(method="bfill"),1)
        
        # 輸出檔案
#         current_path = f"./{product}整併分析資料/"
#         if os.path.exists(current_path) != True:
#             os.mkdir(current_path)
        
#         df_merged.to_csv(f"./{product}整併分析資料/{market}分析資料.csv")
        
    return df_merged

In [4]:
df_test = pd.read_csv("../data/farmproduct_banana_crawler.csv")
df_test

,日期,市場,產品,上價,中價,下價,平均價(元/公斤),平均價_跟前一交易日比較%,交易量(公斤),交易量_跟前一交易日比較%
0,100/01/01,104 台北二,A1 香蕉,35.0,32.1,25.6,31.4,+ 2,9645,- 1
1,100/01/02,104 台北二,A1 香蕉,35.1,33.1,25.8,32,+ 2,12230,+ 27
2,100/01/04,104 台北二,A1 香蕉,35.8,33.1,27.0,32.4,-,11976,-
3,100/01/05,104 台北二,A1 香蕉,36.5,33.9,27.8,33.2,+ 2,7520,- 37
4,100/01/06,104 台北二,A1 香蕉,38.2,35.5,28.9,34.7,+ 7,8160,- 32
...,...,...,...,...,...,...,...,...,...,...
31690,110/06/09,930 台東市,A1 香蕉,28.0,18.3,12.0,21.2,+ 2,2885,- 7
31691,110/06/10,930 台東市,A1 香蕉,24.0,16.8,11.0,18.7,- 12,3885,+ 35
31692,110/06/11,930 台東市,A1 香蕉,26.0,18.3,13.0,20.7,+ 11,4522,+ 16
31693,110/06/12,930 台東市,A1 香蕉,32.0,21.5,15.0,22.8,+ 10,3007,- 34


### 香蕉

In [5]:
# 讀檔路徑
file_path = "../data/farmproduct_{}_crawler.csv"

# 設定參數(產品、同質產品、競爭產品、市場)
product = "banana"
same_type = "scarletbanana"
substitution = "guava"
markets = ['台北二', '台北一', '三重區', '台中市']

# 資料清洗
df = df_cleaner(file_path.format(product))
df_same = df_cleaner(file_path.format(same_type))
df_sub = df_cleaner(file_path.format(substitution))

# 資料合併
output = df_merger(df, df_same, df_sub)

#output

台北二 原始資料筆數(合併後): 3145
台北一 原始資料筆數(合併後): 3145
三重區 原始資料筆數(合併後): 3147
台中市 原始資料筆數(合併後): 3145


### 芭樂

In [6]:
# 讀檔路徑
file_path = "../data/farmproduct_{}_crawler.csv"

# 設定參數(產品、同質產品、競爭產品、市場)
product = "guava"
same_type = "emperorguava"
substitution = "banana"
markets = ['台北二', '台北一', '三重區', '台中市']

# 資料清洗
df = df_cleaner(file_path.format(product))
df_same = df_cleaner(file_path.format(same_type))
df_sub = df_cleaner(file_path.format(substitution))

# 資料合併
df_merger(df, df_same, df_sub)

台北二 原始資料筆數(合併後): 3164
台北一 原始資料筆數(合併後): 3164
三重區 原始資料筆數(合併後): 3164
台中市 原始資料筆數(合併後): 3164


,guava_平均價,guava_交易量,emperorguava_平均價,banana_平均價,guava_前日平均價,guava_5日平均價
日期,,,,,,
2011-01-01,33.7,18212.0,47.7,22.1,33.7,35.5
2011-01-02,36.1,22148.0,44.2,21.7,33.7,35.5
2011-01-03,36.1,22307.5,59.6,23.3,36.1,35.5
2011-01-04,36.1,22467.0,75.0,24.9,36.1,35.5
2011-01-05,35.6,17132.0,54.5,23.6,36.1,35.5
...,...,...,...,...,...,...
2021-07-04,28.6,22960.0,31.8,25.0,26.3,27.2
2021-07-05,28.0,27711.0,35.9,25.0,28.6,27.6
2021-07-06,27.3,32462.0,40.0,25.0,28.0,27.7


## 鳳梨

In [7]:
# 讀檔路徑
file_path = "../data/farmproduct_{}_crawler.csv"

# 設定參數(產品、同質產品、競爭產品、市場)
product = "pineapple"
same_type = "甜蜜蜜"
substitution = "watermelon"
markets = ['台北二', '台北一', '三重區', '台中市']

# 資料清洗
df = df_cleaner(file_path.format(product))
df_same = df_cleaner(file_path.format(same_type))
df_sub = df_cleaner(file_path.format(substitution))

# 資料合併
df_merger(df, df_same, df_sub)

台北二 原始資料筆數(合併後): 3145
台北一 原始資料筆數(合併後): 3145
三重區 原始資料筆數(合併後): 3145
台中市 原始資料筆數(合併後): 3144


,pineapple_平均價,pineapple_交易量,甜蜜蜜_平均價,watermelon_平均價,pineapple_前日平均價,pineapple_5日平均價
日期,,,,,,
2011-01-01,17.0,17027.0,11.0,18.0,17.0,17.4
2011-01-02,17.4,23476.0,11.0,16.4,17.0,17.4
2011-01-03,17.5,37620.0,11.0,14.8,17.4,17.4
2011-01-04,17.6,51764.0,11.0,13.2,17.5,17.4
2011-01-05,17.5,11247.0,11.0,14.1,17.6,17.4
...,...,...,...,...,...,...
2021-06-10,21.0,163380.0,10.0,15.6,21.3,20.8
2021-06-11,21.1,87697.0,5.7,15.2,21.0,21.0
2021-06-12,20.3,82402.0,5.7,15.9,21.1,20.9
